In [2]:
import numpy as np
import sqlite3, time, csv, re
from collections import defaultdict, Counter
from random import choice, randint, shuffle


In [40]:
print('Querying DB...\n')
sql_conn = sqlite3.connect("database.sqlite")
the_data = sql_conn.execute("SELECT subreddit, body FROM May2015 limit 500000")
print('Done querying DB...\n')

Querying DB...

Done querying DB...



In [41]:
print('Building Corpora...\n')
corpus_dict = defaultdict(list)

for post in the_data:
    # corpus.append(post[1])
    corpus_dict[post[0]].append(post[1])
print('Done building Corpora...\n')

Building Corpora...

Done building Corpora...



In [42]:
# separating comments by their subreddits
# 'subreddit1' : ['comment1', 'comment2' 'comment3', etc]
for k in sorted(corpus_dict, key=lambda k: len(corpus_dict[k]), reverse=True):
    print (k, len(corpus_dict[k]))

AskReddit 37018
nfl 35062
nba 15721
hockey 9031
funny 7032
WTF 6025
leagueoflegends 5710
videos 5057
news 4917
movies 4320
worldnews 3991
pics 3947
todayilearned 3415
AdviceAnimals 3057
DotA2 2913
pcmasterrace 2784
SquaredCircle 2548
GlobalOffensive 2476
amiibo 2443
fatpeoplehate 2356
Showerthoughts 2346
anime 2281
politics 2226
trees 2208
KCRoyals 2198
DestinyTheGame 2149
gaming 2145
Browns 2110
eagles 2108
CasualConversation 2107
Texans 2080
Music 2043
IAmA 1977
minnesotavikings 1929
LAClippers 1803
gifs 1777
chicagobulls 1735
electronic_cigarette 1672
relationships 1654
gonewild 1613
smashbros 1517
rangers 1507
thebutton 1444
CHIBears 1434
Fireteams 1375
aww 1345
cowboys 1341
soccer 1298
Mariners 1289
newsokur 1244
TwoXChromosomes 1242
Patriots 1236
csgobetting 1231
explainlikeimfive 1224
greysanatomy 1217
GreenBayPackers 1201
whowouldwin 1197
teenagers 1186
AskWomen 1180
Pokemongiveaway 1176
Random_Acts_Of_Amazon 1175
technology 1175
personalfinance 1155
TumblrInAction 1125
caps 11

In [370]:
# Selecting subreddits

subreds = ['nba']
#subreds = ['leagueoflegends']


#[[all comments in subreddits separated by comma]]
temp = [corpus_dict[a] for a in subreds] 

# [comment1, comment2, etc]
sentences = [item for sublist in temp for item in sublist]

# [c, o, m, m, e, n, t, etc]
characters = [c for sentence in sentences for c in sentence]

In [371]:
# remove tabs/newline??
# remove if exact same comments appear 5+ times
#    '*Please refer to our [detailed rules and posting guidelines.]'
#    'feel free to make a new post that fits within the rules'
# remove links? 

def text_clean(inputlist):
    
    cnt = Counter(inputlist)
    dups = [k for k, v in cnt.items() if v > 5]
    removed = list(set(inputlist) - set(dups))
    
    cleaned = []
    for comment in removed:
        c = re.sub(' +',' ',comment)
        #cleaned.append([a for a in c.replace('&gt;', '>').replace('&lt;', '>').replace('&amp;', '&')]) #.replace("\n", '').replace("\t", '')
        cleaned.append(c.replace('&gt;', '>').replace('&lt;', '>').replace('&amp;', '&'))
    
    return cleaned

print ("original length: " , len(sentences))

cleaned = text_clean(sentences)

print ("cleaned length: " , len(cleaned))

original length:  15721
cleaned length:  14485


In [372]:
cleaned[:100]

['Looked like Achilles :(',
 '"Yah this Supreme Leader. Rodbaby you bad mamba jamba juice! You want me bomb Greece? But I love falafel. Let me call Barack Hussein in Membrane."',
 'Clippers will close it out',
 'thank you diaw. fuck',
 'next time tony runs to foul dj dj should cock back a fist, just to make tony think about for a second. ',
 'God damn he body checked him.',
 '>almost ',
 "I'm not saying that people aren't interested in Soccer, but let's take a look at proportionality. Hockey, Baseball and Basketball are all big sports around the world and in America, we could even include Tennis and Golf in that. However, Soccer is the biggest sports almost everywhere EXCEPT America. We rank Football, Basketball, Hockey and Baseball FAR AND AWAY higher than Soccer.\n\nI'm 22, and I cannot tell you a single time I've ever heard someone say they want to watch an MLS game. People go nuts for the world cup because it occurs once every 4 years; just like I go nuts for table-tennis during th

In [373]:
## combine with dictionary?
def make_ngrams(n, inputlist):

    ngrams = []
    temp_dict = defaultdict(list)

    for comment in inputlist:
        if len(comment) >= n:
            for x in range(0, len(comment)-n):
                
                charlist = []
                
                for i in range(n):
                    charlist.append(comment[x+i])
                
                keys = tuple(charlist[:n-1])
                final_char = charlist[n-1]
                
                temp_dict[keys].append(final_char)
    return temp_dict

N = 5
ngrams = make_ngrams(N, cleaned)

In [374]:
length_cnt = defaultdict(int)

for k, v in ngrams.items():
    length_cnt[len(v)] += 1

length_cnt

defaultdict(int,
            {1: 35912,
             2: 9597,
             3: 4895,
             4: 2982,
             5: 2045,
             6: 1518,
             7: 1227,
             8: 1026,
             9: 827,
             10: 752,
             11: 586,
             12: 523,
             13: 447,
             14: 459,
             15: 369,
             16: 363,
             17: 325,
             18: 312,
             19: 293,
             20: 263,
             21: 256,
             22: 210,
             23: 210,
             24: 219,
             25: 185,
             26: 190,
             27: 167,
             28: 141,
             29: 149,
             30: 156,
             31: 144,
             32: 127,
             33: 117,
             34: 123,
             35: 107,
             36: 86,
             37: 110,
             38: 95,
             39: 103,
             40: 89,
             41: 89,
             42: 74,
             43: 59,
             44: 76,
             45: 88,
 

In [375]:
def filter_dict(threshold, d):
    final_d = {}
    for k, v in d.items():
        if len(v) > threshold:
            final_d[k] = v
        
    return final_d

print('Dictionary length before threhold: ', len(ngrams))

filtered = filter_dict(10, ngrams)

print('Dictionary length after threhold: ', len(filtered))


Dictionary length before threhold:  72165
Dictionary length after threhold:  11384


## Generate random string

In [376]:
def generate(n, ngram_dict, length):
    """Make random text of given length (using ngrams of the given n)."""
    
    #seed_no = randint(0,len(inputlist)) # choose random seed
    
    #start = list(choice(list(ngram_dict))) # start off with randomly chosen n-1 words
    #output = start
    
    # start off with randomly chosen n-1 words that starts with upper case letter
    start = list(choice(tuple([l for l in list(ngram_dict) if l[0].isupper()]))) 
    output = start
    
    letter_count = n-1
    
    while(True):
        
        last_char = output[-1][-1]
        
        # last character was end of sentence punct:
        if (last_char in ['.', '!', '?']):
            if (letter_count > length):
                break
            
        next_key = tuple(output[-(n-1):])
        
        if next_key not in ngram_dict:
            #print ('restart')
            next_key = choice(list(ngram_dict))
        
        output.append(choice(ngram_dict[next_key]))
        
        
        letter_count += 1

    return "".join(output)


## Input starting string

In [74]:
def generate_with_start(startstr, n, ngram_dict, length):
    """Make random text of given length (using ngrams of the given n)."""
    
    # if starting string length > n, use last n-1 
    if len(startstr) >= n:
        last_n_char = startstr[-(n-1):]
        candidates = [ng for ng in list(ngram_dict) if ng[:n-1] == tuple(i for i in last_n_char)]
        
        start = list(choice(candidates))
        output = list(startstr[:len(startstr)-len(last_n_char)]) + start
    else:
        candidates = [ng for ng in list(ngram_dict) if ng[:len(startstr)] == tuple(i for i in startstr)]
            
        start = list(choice(candidates))
        output = start
    
    letter_count = n-1
    
    while(True):
        
        last_char = output[-1][-1]
        
        # last character was end of sentence:
        if (last_char in ['.', '!', '?']):
            if (letter_count > length):
                break
            
        next_key = tuple(output[-(n-1):])
        
        if next_key not in ngram_dict:
            #print ('restart')
            next_key = choice(list(ngram_dict))
        
        output.append(choice(ngram_dict[next_key]))
        
        
        letter_count += 1

    return "".join(output)

In [77]:
startstr = "A"
generate_with_start(startstr, N, filtered, 100)

"Arched it's a humore you did. I'm going rude with they are see my pay thought has crappened came afternations, your mouthen the comple help reappreciationed to my wife, just a literact used tell everyday; hahaaahaneiu]oeeol\ne  v-eo makes in Hispanistral apart of us incredibly Europened to the Lating it she's really his he is 7:th."

## Perplexity -- lower is better

In [ ]:
# Should the corpus be character ngram? or unigram words???

# wrote functions for getting both character level corpus & unigram corpus
# using character level corpus, the probability of 5-gram characters appearing in that corpus is too low
# that when it calculates -log probability later, it gives infinity for almost everything

# switching to word level is a bit better

# when using below functions, use either get_corpus_char, or get_corpus_uni to get all possible tokens from corpus

In [233]:
len(cleaned) # number of comments

33446

In [377]:
# BUILDING CHARACTER LEVEL CORPUS

def char_ngrams(text, n=5):
    return [text[i:i+n] for i in range(len(text)-n+1)]
   
# input: list of comments ['this is comment 1', 'this is comment 2', etc]
# output: list of character level 5-grams ['this ', 'his is', 'is is', 's is ', ' is c', 'is co' , etc]    
def get_corpus_char(all_text):
    all_tokens = []

    for comment in all_text:
        all_tokens += char_ngrams(comment)
    return all_tokens

all_tokens = get_corpus_char(cleaned)

In [378]:
len(all_tokens)

1056116

In [379]:
# BUILDING WORD LEVEL CORPUS

# should this be case-insensitive??
def unigram_words(text):
    return text.split(' ')

# input: list of comments ['this is comment 1', 'this is comment 2', etc]
# output: list of individual words ['this', 'is', 'comment', '1', etc]
def get_corpus_uni(all_text):
    all_tokens = []

    for comment in all_text:
        all_tokens += comment.split()
    return all_tokens

all_tokens = get_corpus_uni(cleaned)

In [380]:
len(all_tokens)

204566

In [383]:
def fivegrams(tokens):
    
    model = defaultdict(lambda: 4.5e-06) # baseline for words that don't appear in corpus -- smoothing
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue

    total = float(sum(model.values()))
    
    for word in model:
        model[word] = model[word]/total
    
    return model

fivegram_prob = fivegrams(all_tokens)

In [384]:
# checking lowest probability word to decide smoothing value above
i = 0
aa = []
lowest = 1
lowest_word = ''
for k, v in fivegram_prob.items():
    if v < lowest:
        lowest = v
        lowest_word = k
        
lowest_word, lowest

('EGordon', 4.888417217739058e-06)

In [385]:
# again, might make everything lowercase when calcualting perplexity
print(fivegram_prob['reddit'])
print(fivegram_prob['Reddit'])

5.866076463730531e-05
4.39955789774236e-05


In [386]:
#computes perplexity of the unigram model on a testset  
def perplexity(testset, model):
    
    testset = testset.split()
    #testset = char_ngrams(testset)
    
    #testset = unigram_words(testset)

    #print(testset)
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
        
    perplexity = pow(perplexity, 1/float(N))
    
    return perplexity

In [341]:
#AR_toeval = []
while(len(AR_toeval) < 2000):
    sent = generate(N, filtered, 12)
    if perplexity(sent, fivegram_prob) < 999999:
        AR_toeval.append(sent)
        print(len(AR_toeval))
    else:
        print ('aa')

703
704
705
706
707
708
709
710
711
712
713
aa
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
aa
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
aa
753
754
755
756
757
aa
758
aa
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
aa
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
aa
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
aa
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
aa
893
894
895
896
897
898
899
900
901
902
903
904
905
aa
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
94

In [343]:
AR_scores_ng = []
for sent in AR_toeval:
    sc = perplexity(sent, fivegram_prob)
    AR_scores_ng.append(sc)
    
np.mean(AR_scores_ng)

39017.467437910105

In [344]:
AR_rnn = []
with open('rnn/AskReddit_rnn.txt', 'r') as f:
    text = f.read()
    print(len(text))

comment = ''  
i = 0
while(i < len(text)):
    
    if (len(comment) < 150):
        comment += text[i]
    else:
        if (text[i] == ' '):
            AR_rnn.append(comment)
            comment = ''
        else:
            comment += text[i]
    i += 1

90001


In [345]:
len(AR_rnn)

583

In [346]:
AR_scores_rnn = []
for sent in NBA_rnn:
    AR_scores_rnn.append(perplexity(sent, fivegram_prob))
    
np.mean(AR_scores_rnn)

16987.940269843937

In [410]:
#NBA_toeval = []
while(len(NBA_toeval) < 2000):
    sent = generate(N, filtered, 12)
    if perplexity(sent, fivegram_prob) < 999999:
        NBA_toeval.append(sent)
    else:
        print ('aa')

KeyboardInterrupt: 

In [412]:
NBA_scores_ng = []
for sent in NBA_toeval:
    NBA_scores_ng.append(perplexity(sent, fivegram_prob))
    
np.mean(NBA_scores_ng)

29822.988635121601

In [389]:
NBA_rnn = []
with open('rnn/nba_rnn.txt', 'r') as f:
    text = f.read()
    print(len(text))

comment = ''   
i = 0
while(i < len(text)):
    
    if (len(comment) < 150):
        comment += text[i]
    else:
        if (text[i] == ' '):
            NBA_rnn.append(comment)
            comment = ''
        else:
            comment += text[i]
    i += 1

89994


In [391]:
NBA_scores_rnn = []
for sent in NBA_rnn:
    NBA_scores_rnn.append(perplexity(sent, fivegram_prob))
    
np.mean(NBA_scores_rnn)

7115.1848627280197

In [366]:
NFL_toeval = []
while(len(NFL_toeval) < 2000):
    sent = generate(N, filtered, 200)
    if perplexity(sent, fivegram_prob) < 999999:
        NFL_toeval.append(sent)
        print(len(NFL_toeval))
    else:
        print ('aa')

In [367]:
NFL_scores_ng = []
for sent in NBA_toeval:
    NFL_scores_ng.append(perplexity(sent, fivegram_prob))
    
np.mean(NFL_scores_ng)

16096.677015308986

In [368]:
NFL_rnn = []
with open('rnn/nfl_rnn.txt', 'r') as f:
    text = f.read()
    print(len(text))

comment = '' 
i = 0
while(i < len(text)):
    
    if (len(comment) < 150):
        comment += text[i]
    else:
        if (text[i] == ' '):
            NFL_rnn.append(comment)
            comment = ''
        else:
            comment += text[i]
    i += 1

90001


In [369]:
NFL_scores_rnn = []
for sent in NFL_rnn:
    NFL_scores_rnn.append(perplexity(sent, fivegram_prob))
    
np.mean(NFL_scores_rnn)

11742.761327920674